In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import pandas as pd
import urllib as url
import json
import requests
from google.colab import files
import io
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk.data
import nltk
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
uploaded = files.upload()

Saving train.csv to train.csv


In [9]:
data= pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [10]:
data.head()

,Column1,Column2
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [11]:
trainingData =data["Column1"]
print(trainingData)

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: Column1, Length: 16000, dtype: object


In [12]:
trainingLabel= data["Column2"]
print(trainingLabel.unique())

['sadness' 'anger' 'love' 'surprise' 'fear' 'joy']


In [13]:
print(trainingLabel.isnull())

0        False
1        False
2        False
3        False
4        False
         ...  
15995    False
15996    False
15997    False
15998    False
15999    False
Name: Column2, Length: 16000, dtype: bool


In [14]:
dic ={}
for i in trainingLabel:
  if i not in dic:
    dic[i]=0
  else:
    dic[i]+=1
print(dic)

{'sadness': 4665, 'anger': 2158, 'love': 1303, 'surprise': 571, 'fear': 1936, 'joy': 5361}


In [15]:
uploaded = files.upload()

Saving testingData.csv to testingData.csv


In [16]:
data= pd.read_csv(io.BytesIO(uploaded['testingData.csv']))

In [17]:
print(data.head())

                                             Column1  Column2
0  im feeling rather rotten so im not very ambiti...  sadness
1          im updating my blog because i feel shitty  sadness
2  i never make her separate from me because i do...  sadness
3  i left with my bouquet of red and yellow tulip...      joy
4    i was feeling a little vain when i did this one  sadness


In [18]:
testingLabel= data["Column2"]
testingData= data["Column1"]
print(testingLabel.head())
print(testingData.head())

0    sadness
1    sadness
2    sadness
3        joy
4    sadness
Name: Column2, dtype: object
0    im feeling rather rotten so im not very ambiti...
1            im updating my blog because i feel shitty
2    i never make her separate from me because i do...
3    i left with my bouquet of red and yellow tulip...
4      i was feeling a little vain when i did this one
Name: Column1, dtype: object


In [19]:
print(testingLabel.isnull())

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Name: Column2, Length: 2000, dtype: bool


In [20]:
'''l = LabelEncoder()
trainingLabel= l.fit_transform(trainingLabel)
testingLabel = l.fit_transform(testingLabel)'''

'l = LabelEncoder()\ntrainingLabel= l.fit_transform(trainingLabel)\ntestingLabel = l.fit_transform(testingLabel)'

In [22]:
print(max(trainingLabel))

surprise


In [23]:
#trainingData= trainingData.str.lower().str.split()
#print(trainingData)

In [24]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
'''def prepare(dataprep):
  ps =PorterStemmer()
  newData =[]
  for i in dataprep:
    temp= []
    for j in i :
      if j not in stopwords:
        temp.append(ps.stem(j))
    newData.append(temp)
  return newData
newTrainingData =pd.Series(prepare(trainingData))'''


'def prepare(dataprep):\n  ps =PorterStemmer()\n  newData =[]\n  for i in dataprep:\n    temp= []\n    for j in i :\n      if j not in stopwords:\n        temp.append(ps.stem(j))\n    newData.append(temp)\n  return newData\nnewTrainingData =pd.Series(prepare(trainingData))'

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [27]:
vocab_size=10000
len_sentence = 150
import re

In [28]:
def textPrepare(data):
    print(data.shape)
    stemmer = PorterStemmer()
    corpus = []
    
    for text in data:
        text = re.sub("[^a-zA-Z]", " ", text)
        
        text = text.lower()
        text = text.split()
        
        text = [stemmer.stem(word) for word in text if word not in stopwords]
        text = " ".join(text)
        
        corpus.append(text)
    oneHotWord = [one_hot(input_text=word, n=vocab_size) for word in corpus]
    embed = pad_sequences(sequences=oneHotWord,
                              maxlen=len_sentence,
                              padding="pre")
    return embed

In [29]:
trainingData =textPrepare(trainingData)
testingData =textPrepare(testingData)


(16000,)
(2000,)


In [30]:
print(trainingData.shape)
print(testingData.shape)

(16000, 150)
(2000, 150)


In [31]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
trainingLabel = np.array(trainingLabel)
trainingLabel = enc.fit_transform(trainingLabel.reshape(-1,1)).toarray()

In [32]:
print(trainingLabel)

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [33]:
testingLabel =np.array(testingLabel)
testingLabel = enc.fit_transform(testingLabel.reshape(-1,1)).toarray()

In [34]:
print(testingLabel)

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [35]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=len_sentence))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation="sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))
model.compile(optimizer="Adam", loss = "categorical_crossentropy", metrics=["accuracy"])

In [36]:
res = model.fit(trainingData, trainingLabel, epochs = 6, batch_size = 64, validation_data=(testingData, testingLabel),verbose = 1)

Epoch 1/6
250/250 [==============================] - 92s 362ms/step - loss: 1.5373 - accuracy: 0.3848 - val_loss: 1.1620 - val_accuracy: 0.5870
Epoch 2/6
250/250 [==============================] - 90s 360ms/step - loss: 0.8012 - accuracy: 0.7281 - val_loss: 0.5167 - val_accuracy: 0.8440
Epoch 3/6
250/250 [==============================] - 90s 359ms/step - loss: 0.3929 - accuracy: 0.8731 - val_loss: 0.3831 - val_accuracy: 0.8660
Epoch 4/6
250/250 [==============================] - 90s 360ms/step - loss: 0.2557 - accuracy: 0.9186 - val_loss: 0.3718 - val_accuracy: 0.8740
Epoch 5/6
250/250 [==============================] - 90s 359ms/step - loss: 0.1907 - accuracy: 0.9358 - val_loss: 0.3681 - val_accuracy: 0.8765
Epoch 6/6
250/250 [==============================] - 90s 358ms/step - loss: 0.1493 - accuracy: 0.9528 - val_loss: 0.3869 - val_accuracy: 0.8765
